In [1]:
import os
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from PIL import Image
import numpy as np


In [2]:
DATASET_PATHS = {
    "real": ["Celeb-real", "YouTube-real"],
    "fake": ["Celeb-Youtube-fake"]
}
TEST_PATH = "test"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [3]:
def extract_frames_from_video(video_path, frame_rate=30):
    frames = []
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame)
            frames.append(pil_img)
        count += 1
    cap.release()
    return frames


def load_data():
    data, labels = [], []
    for label, categories in enumerate(["real", "fake"]):
        for folder in DATASET_PATHS[categories]:
            folder_path = os.path.join(folder)
            for filename in os.listdir(folder_path):
                if filename.endswith(('.mp4', '.avi', '.mov')):
                    video_path = os.path.join(folder_path, filename)
                    frames = extract_frames_from_video(video_path)
                    for frame in frames[:5]:  # use only first 5 frames
                        try:
                            tensor = transform(frame)
                            data.append(tensor)
                            labels.append(label)
                        except:
                            continue
    return torch.stack(data), torch.tensor(labels)


In [4]:
print("Loading and preprocessing videos...")
X, y = load_data()
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.3)


Loading and preprocessing videos...


In [5]:
BATCH_SIZE = 16

train_dataset = TensorDataset(train_x, train_y)
test_dataset = TensorDataset(test_x, test_y)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)


In [6]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 2)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)


c:\Users\ASHIRWAD\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ASHIRWAD\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
print("Training model...")
for epoch in range(5):
    model.train()
    epoch_loss = 0
    for batch_x, batch_y in zip(train_x.split(BATCH_SIZE), train_y.split(BATCH_SIZE)):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/5, Loss: {epoch_loss / len(train_x):.4f}")


Training model...
Epoch 1/5, Loss: 0.0176
Epoch 2/5, Loss: 0.0086
Epoch 3/5, Loss: 0.0050
Epoch 4/5, Loss: 0.0038
Epoch 5/5, Loss: 0.0034


In [8]:
torch.save(model.state_dict(), "model.pth")
print("Model saved as 'model.pth'.")


Model saved as 'model.pth'.


In [9]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch_x, batch_y in test_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        preds = model(batch_x)
        pred_labels = torch.argmax(preds, dim=1)
        correct += (pred_labels == batch_y).sum().item()
        total += batch_y.size(0)

print(f"Validation Accuracy: {correct / total * 100:.2f}%")


Validation Accuracy: 90.73%


In [10]:
def predict_video(video_path):
    print(f"\nPredicting: {video_path}")
    frames = extract_frames_from_video(video_path)
    if not frames:
        print("No frames found.")
        return
    model.eval()
    results = []
    for frame in frames[:10]:
        img_tensor = transform(frame).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(img_tensor)
            pred = torch.argmax(output, 1).item()
            results.append(pred)
    real_pct = results.count(0) / len(results) * 100
    fake_pct = results.count(1) / len(results) * 100
    print(f"Real: {real_pct:.2f}%, Fake: {fake_pct:.2f}% => {'Real' if real_pct > fake_pct else 'Fake'}")


In [13]:
predict_video(r"E:\12345678\test\id3_id6_0001.mp4")  



Predicting: E:\12345678\test\id3_id6_0001.mp4
Real: 10.00%, Fake: 90.00% => Fake
